In [ ]:
pip install tink

In [384]:
import io
import base64
import json

import tink
from tink import aead
from tink import cleartext_keyset_handle

aead.register()

def create_new_kek():
    
    kh_key = tink.new_keyset_handle(aead.aead_key_templates.AES256_GCM)
    buffer = io.BytesIO()
    writer = tink.BinaryKeysetWriter(buffer)
    cleartext_keyset_handle.write(writer, kh_key)
    buffer.seek(0)
    return base64.b64encode(buffer.read())

#def create_new_dek():

#def get_dek():

def ins_table_kek(i_table, i_kek, i_table_kek):
    import google.cloud.bigquery.dbapi as bq
    from datetime import datetime

    dttm = datetime.now()
    rec_id = dttm.strftime("%Y%m%d%H%M%S")
    kek_str = str(i_kek, 'utf-8')

    ins_query = 'insert into `' + i_table_kek + '` (table_id, table_nm, kek) values '
    ins_query = ins_query + '(' + rec_id + ', \'' + i_table + '\', FROM_BASE64(\'' + kek_str + '\'))'
    
    con = bq.connect()
    cursor = con.cursor()
    cursor.execute(ins_query)
    con.commit()
    con.close()

    
def upd_table_kek(i_table, i_kek, i_table_kek):
    import google.cloud.bigquery.dbapi as bq
    from datetime import datetime

    dttm = datetime.now()
    rec_id = dttm.strftime("%Y%m%d%H%M%S")
    kek_str = str(i_kek, 'utf-8')

    upd_query = 'update `' + i_table_kek + '`'
    upd_query = upd_query + ' set kek = FROM_BASE64(\'' + kek_str + '\')'
    upd_query = upd_query + ' where table_nm = \'' + i_table + '\''
    
    con = bq.connect()
    cursor = con.cursor()
    cursor.execute(upd_query)
    con.commit()
    con.close()


def ins_table_dek(i_table, i_table_dek, i_table_kek):
    from google.cloud import bigquery
    import pandas
    
    kek_km_df = get_table_kek(i_table, i_table_kek)
    clear_dek = create_new_kek()
    clear_dek_str = str(clear_dek, 'utf-8')
    wrapped_dek = encrypt_text(kek_km_df, clear_dek_str)
    
    client = bigquery.Client(location="US")
    
    sel_query = 'select table_id from `' + i_table_kek + '`'
    sel_query = sel_query + ' where table_nm = \'' + i_table + '\''
    query_job = client.query(
    sel_query)  # API request - starts the query
    df = query_job.to_dataframe()
    table_id_df = df['table_id']
    table_id = table_id_df.values[0]
    
    wrapped_dek_str = str(wrapped_dek, 'utf-8')
    
    ins_query = 'insert into `' + i_table_dek + '` (table_id, table_nm, wrapped_dek) values '
    ins_query = ins_query + '(' + str(table_id) + ', \'' + i_table + '\', FROM_BASE64(\'' + wrapped_dek_str + '\'))'
    
    con = bq.connect()
    cursor = con.cursor()
    cursor.execute(ins_query)
    con.commit()
    con.close()
    
#def upd_table_dek(i_table, i_kek, i_table_dek):

def get_table_kek(i_table, i_table_kek):
    from google.cloud import bigquery
    import pandas
    client = bigquery.Client(location="US")
    
    sel_query = 'select kek, table_nm from `' + i_table_kek + '`'
    sel_query = sel_query + ' where table_nm = \'' + i_table + '\''
    
    query_job = client.query(
    sel_query,
    # Location must match that of the dataset(s) referenced in the query.
    location="US",
    )  # API request - starts the query

    df = query_job.to_dataframe()
    return df

def get_table_dek(i_table, i_table_dek, i_table_kek):
    from google.cloud import bigquery
    import pandas

    i_table_kek= 'gcp-rrk-001.sandbox.t_crypto_kek'
    i_table_dek= 'gcp-rrk-001.sandbox.t_crypto_dek'
    i_table = 't_pci_data_set3'

    kek_km_df = get_table_kek(i_table, i_table_kek)

    client = bigquery.Client(location="US")    
    sel_query = 'select wrapped_dek, table_nm from `' + i_table_dek + '`'
    sel_query = sel_query + ' where table_nm = \'' + i_table + '\''
    query_job = client.query(sel_query)  # API request - starts the query

    dek_km_df = query_job.to_dataframe()
    dek_df = dek_km_df['wrapped_dek']
    wrapped_dek = dek_df.values[0]
    clear_dek = decrypt_cypher(kek_km_df, base64.b64encode(wrapped_dek))

    dek_km_df.rename(columns = {'wrapped_dek':'kek'}, inplace = True)
    dek_km_df.at[0,'kek'] = base64.b64decode(clear_dek)
    return dek_km_df

def encrypt_text(i_key_material_df, i_cleartext):
    import base64
    import tink
    from tink import aead
    from tink import cleartext_keyset_handle

    aead.register()
    kek_df = i_key_material_df['kek']
    table_df = i_key_material_df['table_nm']
    
    kek = kek_df.values[0]
    table_nm = table_df.values[0]
    
    plaintext = i_cleartext.encode('utf-8')
    associated_data = table_nm.encode('utf-8')

    kh_kek = cleartext_keyset_handle.read(tink.BinaryKeysetReader(kek))
    aead_primitive = kh_kek.primitive(aead.Aead)
    return base64.b64encode(aead_primitive.encrypt(plaintext, associated_data))
    #return cipher_text 
    #return aead_primitive.encrypt(plaintext, associated_data)
    
def decrypt_cypher(i_key_material_df, i_cyphertext):
    import base64
    import tink
    from tink import aead
    from tink import cleartext_keyset_handle

    aead.register()
    kek_df = i_key_material_df['kek']
    table_df = i_key_material_df['table_nm']
    
    kek = kek_df.values[0]
    table_nm = table_df.values[0]
    associated_data = table_nm.encode('utf-8')

    kh_kek = cleartext_keyset_handle.read(tink.BinaryKeysetReader(kek))
    aead_primitive = kh_kek.primitive(aead.Aead)
    return aead_primitive.decrypt(base64.b64decode(i_cyphertext), associated_data)
    #cleartext = aead_primitive.decrypt(base64.b64decode(i_cyphertext), associated_data)
    #return str(cleartext, 'utf-8')
    


In [364]:
i_table_kek= 'gcp-rrk-001.sandbox.t_crypto_kek'
i_table = 't_pci_data_set4'
i_kek = create_new_kek()
ins_table_kek(i_table, i_kek, i_table_kek)

In [261]:
i_table_kek= 'gcp-rrk-001.sandbox.t_crypto_kek'
i_table = 't_pci_data_set4'
kek = get_table_kek(i_table, i_table_kek)
print(kek)

                                                 kek         table_nm
0  b'\x08\xe2\xbd\xca\xfe\x02\x12d\nX\n0type.goog...  t_pci_data_set4


In [316]:
i_table_kek= 'gcp-rrk-001.sandbox.t_crypto_kek'
i_table = 't_pci_data_set4'
i_kek = create_new_kek()
upd_table_kek(i_table, i_kek, i_table_kek)
kek = get_table_kek(i_table, i_table_kek)
print(kek)

                                                 kek         table_nm
0  b'\x08\x93\xc6\x80\xe3\x07\x12d\nX\n0type.goog...  t_pci_data_set4


In [385]:
i_table_kek= 'gcp-rrk-001.sandbox.t_crypto_kek'
i_table_dek= 'gcp-rrk-001.sandbox.t_crypto_dek'
i_table = 't_pci_data_set3'
dek = get_table_dek(i_table, i_table_dek, i_table_kek)
print(dek)

                                                 kek         table_nm
0  b'\x08\xde\x96\x84\xd1\x02\x12d\nX\n0type.goog...  t_pci_data_set3


In [349]:
#encrypt values and load to bigquery

import pandas

i_table_kek= 'gcp-rrk-001.sandbox.t_crypto_kek'
i_table_nm = 't_pci_data_set2'
tgt_table = 'gcp-rrk-001.sandbox.t_sample_customer'

km_df = get_table_kek(i_table_nm, i_table_kek)

name = 'rohan ramnarine k'
ssn = '555-33-5555'

wrapped_ssn = encrypt_text(km_df, ssn)
print('cypher:', ssn)

df = pandas.DataFrame([{'name':name, 'ssn':ssn, 'wrapped_ssn':str(wrapped_ssn, 'utf-8')}])

job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",
)

job = client.load_table_from_dataframe(
    df, tgt_table, job_config
)  # Make an API request.
job.result()  # Wait for the job to complete.

table = client.get_table(tgt_table)  # Make an API request.
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), tgt_table
    )
)


cypher: 555-33-5555
Loaded 1 rows and 3 columns to gcp-rrk-001.sandbox.t_sample_customer


In [354]:
query = """
    SELECT *
    FROM `gcp-rrk-001.sandbox.t_sample_customer`
"""
query_job = client.query(
    query,
    # Location must match that of the dataset(s) referenced in the query.
    location="US",
)  # API request - starts the query

df = query_job.to_dataframe()
df
wrapped_ssn_df = df['wrapped_ssn']
wrapped_ssn = wrapped_ssn_df.values[0]
print(wrapped_ssn)
print(wrapped_ssn.encode('utf-8'))


cyphertext_t = wrapped_ssn.encode('utf-8')
i_table_kek= 'gcp-rrk-001.sandbox.t_crypto_kek'
i_table_nm = 't_pci_data_set2'
km_df = get_table_kek(i_table_nm, i_table_kek)
cleartext = decrypt_cypher(km_df, cyphertext_t)
print('clear text:', cleartext)

AT+pN3EGtyOfKnz8aYKrvV6D963FroZgaRirs6y1xu2ROHwkyyRVyOWLQcg=
b'AT+pN3EGtyOfKnz8aYKrvV6D963FroZgaRirs6y1xu2ROHwkyyRVyOWLQcg='
clear text: b'555-33-5555'


In [321]:
cyphertext_t = b'AXxgIxMiwQyl2f7Fgm2zcWaUDPMOIKANYcfE1TelkXDT7qA25U0='
i_table_kek= 'gcp-rrk-001.sandbox.t_crypto_kek'
i_table_nm = 't_pci_data_set4'
km_df = get_table_kek(i_table_nm, i_table_kek)
cleartext = decrypt_cypher(km_df, cyphertext_t)
print('clear text:', cleartext)

clear text: Rohan


In [236]:
  # Initialise Tink

aead.register()
key_template = aead.aead_key_templates.AES128_GCM
keyset_handle = tink.KeysetHandle.generate_new(key_template)

key_file_path = 'test1.json'
with open(key_file_path, 'wt') as keyset_file:
    cleartext_keyset_handle.write(
    tink.JsonKeysetWriter(keyset_file), keyset_handle)
    
out = io.BytesIO()
writer = tink.BinaryKeysetWriter(out)
cleartext_keyset_handle.write(writer, keyset_handle)
out.seek(0)

key_bin = out.read()
key_bin_encode = base64.b64encode(key_bin)
key_bin_decode = base64.b64decode(key_bin_encode)

print(key_bin)
print(key_bin_encode)
print(key_bin_decode)


b'\x08\x86\xae\xa1\x84\x07\x12T\nH\n0type.googleapis.com/google.crypto.tink.AesGcmKey\x12\x12\x1a\x10v\x93\\2\xab\xb5\xdb\xcc\x13\xb7\x84\xc6\x8d\xb9\x80Y\x18\x01\x10\x01\x18\x86\xae\xa1\x84\x07 \x01'
b'CIauoYQHElQKSAowdHlwZS5nb29nbGVhcGlzLmNvbS9nb29nbGUuY3J5cHRvLnRpbmsuQWVzR2NtS2V5EhIaEHaTXDKrtdvME7eExo25gFkYARABGIauoYQHIAE='
b'\x08\x86\xae\xa1\x84\x07\x12T\nH\n0type.googleapis.com/google.crypto.tink.AesGcmKey\x12\x12\x1a\x10v\x93\\2\xab\xb5\xdb\xcc\x13\xb7\x84\xc6\x8d\xb9\x80Y\x18\x01\x10\x01\x18\x86\xae\xa1\x84\x07 \x01'


In [332]:
import io
import base64
import json

import tink
from tink import aead
from tink import cleartext_keyset_handle

aead.register()


i_table_kek= 'gcp-rrk-001.sandbox.t_crypto_kek'
i_table = 't_pci_data_set2'
km_df = get_table_kek(i_table, i_table_kek)
kek_df = km_df['kek']
kek = kek_df.values[0]

plaintext = b'Rohan'
associated_data = b'Ramnarine'
print(base64.b64encode(kek))

kh_kek = cleartext_keyset_handle.read(tink.BinaryKeysetReader(kek))

# 2. Get the primitive.
aead_primitive = kh_kek.primitive(aead.Aead)

ciphertext = aead_primitive.encrypt(plaintext, associated_data)

print(ciphertext)

temp = base64.b64encode(ciphertext)

print(temp)

print(base64.b64decode(temp))

cleartext = aead_primitive.decrypt(base64.b64decode(temp), associated_data)

print(cleartext)

cleartext1 = aead_primitive.decrypt(base64.b64decode('AS/SnuJTIztGQqyDi2L/RB/bgueSNQT4gBcIlFPuOK44IBZQvzg='), associated_data)

print(cleartext1)

b'COrYutICEmQKWAowdHlwZS5nb29nbGVhcGlzLmNvbS9nb29nbGUuY3J5cHRvLnRpbmsuQWVzR2NtS2V5EiIaIAz8v52Nu3L9D7fB3Glzhtlc9Bn6bVz58fDIAAWQswfXGAEQARjq2LrSAiAB'
b'\x01*N\xacj\x83#\xbb(+\x83\x94\x0f\x19\x94\xfc\xe1X\xc4\xe7\xc1\xe2\xda\x16\xb4\x98W0\xdd@<0*7`\xbf\xd7\xd4'
b'ASpOrGqDI7soK4OUDxmU/OFYxOfB4toWtJhXMN1APDAqN2C/19Q='
b'\x01*N\xacj\x83#\xbb(+\x83\x94\x0f\x19\x94\xfc\xe1X\xc4\xe7\xc1\xe2\xda\x16\xb4\x98W0\xdd@<0*7`\xbf\xd7\xd4'
b'Rohan'


TinkError: Decryption failed.

In [263]:
import google.cloud.bigquery.dbapi as bq

from datetime import datetime
dttm = datetime.now()
i_table_kek= 'gcp-rrk-001.sandbox.t_crypto_kek'
i_table_nm = 't_pci_data_set5'

rec_id = dttm.strftime("%Y%m%d%H%M%S")
kek_str = str(create_new_kek(), 'utf-8')

ins_query = 'insert into `' + i_table_kek + '` (table_id, table_nm, kek) values '
ins_query = ins_query + '(' + rec_id + ', \'' + i_table_nm + '\', FROM_BASE64(\'' + kek_str + '\'))'

print (ins_query)

con = bq.connect()
cursor = con.cursor()
cursor.execute(ins_query)
con.commit()
con.close()


insert into `gcp-rrk-001.sandbox.t_crypto_kek` (table_id, table_nm, kek) values (20210330211507, 't_pci_data_set5', FROM_BASE64('CM6nwqABEmQKWAowdHlwZS5nb29nbGVhcGlzLmNvbS9nb29nbGUuY3J5cHRvLnRpbmsuQWVzR2NtS2V5EiIaICmJ4N7vYLY0/UuHChDfuR35LqmeUoevW4PpIQlMwuhTGAEQARjOp8KgASAB'))


In [ ]:
from datetime import datetime
import time
dttm = datetime.now()
timestampStr = dttm.strftime("%Y%m%d%H%M%S")
print(timestampStr)

In [383]:

from google.cloud import bigquery
import pandas

i_table_kek= 'gcp-rrk-001.sandbox.t_crypto_kek'
i_table_dek= 'gcp-rrk-001.sandbox.t_crypto_dek'
i_table = 't_pci_data_set3'

kek_km_df = get_table_kek(i_table, i_table_kek)

client = bigquery.Client(location="US")    
sel_query = 'select wrapped_dek, table_nm from `' + i_table_dek + '`'
sel_query = sel_query + ' where table_nm = \'' + i_table + '\''
query_job = client.query(sel_query)  # API request - starts the query

dek_km_df = query_job.to_dataframe()
dek_df = dek_km_df['wrapped_dek']
wrapped_dek = dek_df.values[0]
clear_dek = decrypt_cypher(kek_km_df, base64.b64encode(wrapped_dek))

dek_km_df.rename(columns = {'wrapped_dek':'kek'}, inplace = True)
dek_km_df.at[0,'kek'] = base64.b64decode(clear_dek)
return dek_km_df




wrapped dek b'AXbOTa8IkN90OO3SkG3qNX8GyJH+aaEqbVAdAw0AE33RgYx8LfAemWhCUeEU62G1B3ZuTyuciFnJCK/fmuo16CUhmHsWLopy7zdZV0fjVAT/X0c1uwKR2I4Agfy04ftiUEhcMnwob4GZpIKIIMtriN+8ssCdPgo4HzLt64rrg0eU0hZiyhU5NvVLWFR/o5L1nkhnseTACYS9u8H/zibBJMdn35kvKHvj366CsKzFSKEn'
clear dek b'Q042V2hORUNFbVFLV0Fvd2RIbHdaUzVuYjI5bmJHVmhjR2x6TG1OdmJTOW5iMjluYkdVdVkzSjVjSFJ2TG5ScGJtc3VRV1Z6UjJOdFMyVjVFaUlhSUZjSnJhRllWc1FYQmpZRmdyMndERDZMV056SDJPS01LcEIyUmZnQ3Z4TUFHQUVRQVJqZWxvVFJBaUFC'
                                         wrapped_dek         table_nm
0  b"\x01v\xceM\xaf\x08\x90\xdft8\xed\xd2\x90m\xe...  t_pci_data_set3
                                                 kek         table_nm
0  b'\x08\xde\x96\x84\xd1\x02\x12d\nX\n0type.goog...  t_pci_data_set3
                                                 kek         table_nm
0  b'\x08\xaf\x9b\xb9\xb6\x07\x12d\nX\n0type.goog...  t_pci_data_set3


In [ ]:
print(base64.b64decode(kek_str.encode('utf-8')))

In [ ]:
kek_df = df['kek']

In [ ]:
print(kek_df.values[0])

In [336]:
    from datetime import datetime

    dttm = datetime.now()
    rec_id = dttm.strftime("%Y%m%d%H%M%S")
    print(rec_id)

20210331000504


In [342]:
import pandas

pi  = 3.142
e   = 2.718
phi = 1.618

#df = pandas.DataFrame([[pi,e,phi]],columns=['pi','e','phi'])
#df

df = pandas.DataFrame([{'pi':pi, 'e':e, 'phi':phi}])
df

,pi,e,phi
0,3.142,2.718,1.618


In [ ]:
SELECT tgt, SAFE_CONVERT_BYTES_TO_STRING(AEAD.DECRYPT_BYTES(kek.kek, FROM_BASE64(tgt.wrapped_ssn), CAST(kek.table_nm AS BYTES))) clear_ssn
FROM `gcp-rrk-001.sandbox.t_crypto_kek` kek
cross join `gcp-rrk-001.sandbox.t_sample_customer` tgt
where kek.table_nm = 't_pci_data_set2';

In [370]:

    from google.cloud import bigquery
    import pandas
    
    i_table_kek= 'gcp-rrk-001.sandbox.t_crypto_kek'
    i_table_dek= 'gcp-rrk-001.sandbox.t_crypto_dek'
    i_table = 't_pci_data_set4'
    
    kek_km_df = get_table_kek(i_table, i_table_kek)
    clear_dek = create_new_kek()
    clear_dek_str = str(clear_dek, 'utf-8')
    wrapped_dek = encrypt_text(kek_km_df, clear_dek_str)
    
    client = bigquery.Client(location="US")
    
    sel_query = 'select table_id from `' + i_table_kek + '`'
    sel_query = sel_query + ' where table_nm = \'' + i_table + '\''
    query_job = client.query(
    sel_query)  # API request - starts the query
    df = query_job.to_dataframe()
    table_id_df = df['table_id']
    table_id = table_id_df.values[0]
    
    wrapped_dek_str = str(wrapped_dek, 'utf-8')
    

    #ins_query = 'insert into `' + i_table_dek + '` (table_id, table_nm, wrapped_dek, clear_dek) values '
    #ins_query = ins_query + '(' + str(table_id) + ', \'' + i_table + '\', FROM_BASE64(\'' + wrapped_dek_str + '\'), FROM_BASE64(\'' + clear_dek_str + '\'))'

    ins_query = 'insert into `' + i_table_dek + '` (table_id, table_nm, wrapped_dek) values '
    ins_query = ins_query + '(' + str(table_id) + ', \'' + i_table + '\', FROM_BASE64(\'' + wrapped_dek_str + '\'))'
    
    con = bq.connect()
    cursor = con.cursor()
    cursor.execute(ins_query)
    con.commit()
    con.close()

    